In [ ]:
import torch
from watermark import watermark
from shared_utilities import PyTorchMLP, MNISTDataModule, LightningModel
import lightning as L

In [16]:
print(watermark(packages='torch,lightning', python=True))
print(f"Torch CUDA available? {torch.cuda.is_available()}")

Python implementation: CPython
Python version       : 3.11.6
IPython version      : 9.4.0

torch    : 2.2.2
lightning: 2.5.2

Torch CUDA available? False


In [ ]:
dm = MNISTDataModule()

pytorch_model = PyTorchMLP(num_features=784, num_classes=10)
lighting_model = LightningModel(model=pytorch_model, learning_rate=0.05)

In [ ]:
from lightning.pytorch.loggers import CSVLogger

In [ ]:
trainer = L.Trainer(
    max_epochs=15,
    accelerator='auto',
    devices='auto',
    deterministic=True, # this the random seed constant setting.
    logger=CSVLogger(save_dir='logs/',name='my-model')
)
trainer.fit(lighting_model,datamodule=dm)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt


metrics = pd.read_csv(f"{trainer.logger.log_dir}/metrics.csv")

aggreg_metrics = []
agg_col = "epoch"
for i, dfg in metrics.groupby(agg_col):
    agg = dict(dfg.mean())
    agg[agg_col] = i
    aggreg_metrics.append(agg)

df_metrics = pd.DataFrame(aggreg_metrics)
df_metrics[["Training loss", "val loss"]].plot(
    grid=True, legend=True, xlabel="Epoch", ylabel="Loss"
)

df_metrics[["train accuracy", "val accuracy"]].plot(
    grid=True, legend=True, xlabel="Epoch", ylabel="ACC"
)

plt.show()

In [ ]:
df_metrics

In [ ]:
trainer.save_checkpoint('model.ckpt')

In [ ]:
LightningModel

# C 6-1 Model checkpointing

In [ ]:
dm = MNISTDataModule()

pytorch_model = PyTorchMLP(num_features=784, num_classes=10)
lighting_model = LightningModel(model=pytorch_model, learning_rate=0.05)

In [ ]:
from lightning.pytorch.callbacks import ModelCheckpoint

callbacks = [
    ModelCheckpoint(save_top_k=1,monitor='val_loss',mode='min',save_last=True)
]

In [ ]:
trainer = L.Trainer(
    callbacks=callbacks,
    max_epochs=15,
    accelerator='auto',
    devices='auto',
    deterministic=True, # this the random seed constant setting.
    logger=CSVLogger(save_dir='logs/',name='my-model')
)
trainer.fit(lighting_model,datamodule=dm)

In [ ]:
# checkpoint tip

path = trainer.checkpoint_callback.best_model_path
path = trainer.checkpoint_callback.last_model_path
print(path)

lighting_model = LightningModel.load_from_checkpoint(
    path,
    learning_rate=0.05,
    model=pytorch_model,
)
lighting_model.eval();

In [ ]:
# trainer.test(model )

# Setting Learingin rate


In [17]:
trainer = L.Trainer(
    max_epochs=15,
    auto_lr_find=True,
    accelerator='auto',
    devices='auto',
    deterministic=True, # this the random seed constant setting.
    logger=CSVLogger(save_dir='logs/',name='my-model')
)
results = trainer.tune(lighting_model,datamodule=dm)

TypeError: Trainer.__init__() got an unexpected keyword argument 'auto_lr_find'

In [ ]:
fig = results['L']